How to run this

You need to upload this file and Samples.zip to google drive, (/drive/myDrvive) when drive is mounted, it will unzip files into collab contents.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import os.path
from pathlib import Path
import glob
from PIL import Image
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import keras
# from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import regularizers
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
from keras.optimizers import RMSprop,Adam,Optimizer,Optimizer
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D,BatchNormalization,\
                        Permute, TimeDistributed, Bidirectional,GRU, SimpleRNN, LSTM, GlobalAveragePooling2D, SeparableConv2D
from keras import models
from keras import layers
import tensorflow as tf
from keras.applications import VGG16,VGG19,inception_v3
from keras import backend as K


from zipfile import ZipFile as zf, BadZipFile
import shutil
import random

import cv2
from math import floor

from tqdm import tqdm
from time import time

from google.colab import files


!pip install git+https://github.com/openai/CLIP.git > /dev/null #run only once

import torch
from torchvision import transforms
import clip  # this loads OpenAI's CLIP model

import joblib
from torch import nn

  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-l5evacma


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Ensure the file path is correct and the file exists
file_path = "/content/drive/MyDrive/Samples.zip"

# Add error handling to provide more context
try:
    with zf(file_path, "r") as zip_ref:
        zip_ref.extractall("/")
except FileNotFoundError:
    print(f"Error: File not found at path: {file_path}")
except BadZipFile:
    print(f"Error: Invalid or corrupted zip file at path: {file_path}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:

#clean up helper fn

def clean_up():
    folder_path = "/content/purple_cells"  # or any folder name
    files = glob.glob(f"{folder_path}/*")
    for file in files:
        os.remove(file)

clean_up()


In [ ]:
from tensorflow.keras.models import load_model
#model = load_model("/content/leukemia_model_vf.keras")  # change filename as needed

# Load both models
model_eff = load_model("/content/drive/MyDrive/efficientnetv2_leukemia.keras")
model_inc = load_model("/content/drive/MyDrive/inceptionv3_leukemia.keras")


In [ ]:
def normalize_lighting(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    h_, w_ = v.shape
    corners = np.concatenate([
        v[0:50, 0:50].flatten(),
        v[0:50, w_-50:w_].flatten(),
        v[h_-50:h_, 0:50].flatten(),
        v[h_-50:h_, w_-50:w_].flatten()
    ])
    bg_val = int(np.median(corners))
    target_bg = 230
    print(bg_val)
    if bg_val < target_bg - 10:
        delta = target_bg - bg_val
    else:
        delta = 0  # no need to brighten if already bright

    v_int = v.astype(np.int16)
    v_new = np.clip(v_int + delta, 0, 255).astype(np.uint8)

    hsv_normalized = cv2.merge([h, s, v_new])
    return cv2.cvtColor(hsv_normalized, cv2.COLOR_HSV2BGR)

def fn_patching(image_path):
    image = cv2.imread(image_path)
    image = normalize_lighting(image)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Color detection with slightly widened ranges
    mask1 = cv2.inRange(hsv, np.array([115, 35, 35]), np.array([140, 255, 255]))
    mask2 = cv2.inRange(hsv, np.array([115, 40, 70]), np.array([175, 150, 255]))
    main_mask = cv2.bitwise_or(mask1, mask2)

    # Strict exclusion for bright non-cells
    exclude_lower = np.array([118, 20, 220])
    exclude_upper = np.array([130, 70, 255])
    exclude_mask = cv2.inRange(hsv, exclude_lower, exclude_upper)
    mask = cv2.bitwise_and(main_mask, cv2.bitwise_not(exclude_mask))

    # Morphological processing
    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.dilate(mask, kernel, iterations=1)

    # Watershed segmentation
    contours_pre, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    sure_bg = cv2.dilate(mask, kernel, iterations=3)
    dist_transform = cv2.distanceTransform(mask, cv2.DIST_L2, 5)
    ret, sure_fg = cv2.threshold(dist_transform, 0.25 * dist_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)

    ret, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1
    markers[unknown == 255] = 0

    wshed_img = image.copy()
    markers = cv2.watershed(wshed_img, markers)
    wshed_mask = np.zeros_like(mask)
    wshed_mask[markers > 1] = 255

    contours_post, _ = cv2.findContours(wshed_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Initial contour filtering
    final_contours = []
    for contour in contours_post:
        if len(contour) < 5:
            continue

        area = cv2.contourArea(contour)
        if area < 400 or area > 100000:
            print(area)
            continue

        perimeter = cv2.arcLength(contour, True)
        if perimeter == 0:
            continue

        circularity = 4 * np.pi * (area / (perimeter ** 2))
        if circularity < 0.2:
            print(circularity)
            continue

        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = float(w) / h
        if aspect_ratio < 0.5 or aspect_ratio > 3.0:
            continue

        hull = cv2.convexHull(contour)
        hull_area = cv2.contourArea(hull)
        if hull_area == 0:
            continue

        mask_roi = np.zeros(mask.shape, dtype=np.uint8)
        cv2.drawContours(mask_roi, [contour], -1, 255, -1)
        mean_val = cv2.mean(image, mask=mask_roi)
        blue, green, red = mean_val[:3]
        if not (blue > red + 8 and blue > green - 15):
            continue

        final_contours.append(contour)

    # Add back any valid pre-watershed contours
    for c in contours_pre:
        if len(c) < 5:
            continue

        area = cv2.contourArea(c)
        if area < 600 or area > 120000:
            #if area > 80000:
                #print(area)
            continue

        perimeter = cv2.arcLength(c, True)
        if perimeter == 0:
            continue

        circularity = 4 * np.pi * (area / (perimeter ** 2))
        if circularity < 0.4:
            continue

        x, y, w, h = cv2.boundingRect(c)
        aspect_ratio = float(w) / h
        if aspect_ratio < 0.5 or aspect_ratio > 2.8:
            print(aspect_ratio)
            continue

        hull = cv2.convexHull(c)
        hull_area = cv2.contourArea(hull)
        if hull_area == 0:
            print("hull")
            continue

        mask_roi = np.zeros(mask.shape, dtype=np.uint8)
        cv2.drawContours(mask_roi, [c], -1, 255, -1)
        mean_val = cv2.mean(image, mask=mask_roi)
        blue, green, red = mean_val[:3]
        if not (blue > red + 8 and blue > green - 15):
            continue

        unique = True
        for existing in final_contours:
            if cv2.matchShapes(existing, c, cv2.CONTOURS_MATCH_I2, 0) < 0.5:
                unique = False
                break
        if unique:
            final_contours.append(c)

    # Bounding box containment handling
    filtered_boxes = []
    boxes = [cv2.boundingRect(c) for c in final_contours]
    contours = list(zip(boxes, final_contours))

    contours.sort(key=lambda x: x[0][2]*x[0][3], reverse=True)

    while contours:
        current_box, current_contour = contours.pop(0)
        x1, y1, w1, h1 = current_box
        area1 = w1 * h1

        keep_current = True
        contained_contours = []

        for other_box, other_contour in contours[:]:
            x2, y2, w2, h2 = other_box
            area2 = w2 * h2

            x_left = max(x1, x2)
            y_top = max(y1, y2)
            x_right = min(x1 + w1, x2 + w2)
            y_bottom = min(y1 + h1, y2 + h2)

            if x_right < x_left or y_bottom < y_top:
                continue

            intersection_area = (x_right - x_left) * (y_bottom - y_top)

            if intersection_area >= 0.9 * area2:
                contained_contours.append(other_contour)
                contours.remove((other_box, other_contour))

            elif intersection_area >= 0.9 * area1:
                keep_current = False
                break

        if keep_current:
            if contained_contours:
                if len(contained_contours) >= 1:
                    filtered_boxes.extend([(cv2.boundingRect(c), c) for c in contained_contours])
                else:
                    filtered_boxes.append((cv2.boundingRect(contained_contours[0]), contained_contours[0]))
            else:
                filtered_boxes.append((current_box, current_contour))

    final_contours = [contour for (box, contour) in filtered_boxes]

    # Output results
    os.makedirs("purple_cells", exist_ok=True)
    annotated = image.copy()
    cell_num = 0
    scale = 1.3

    for contour in final_contours:
        x, y, w, h = cv2.boundingRect(contour)
        cx = x + w // 2
        cy = y + h // 2
        new_w = int(w * scale)
        new_h = int(h * scale)
        new_x = max(cx - new_w // 2, 0)
        new_y = max(cy - new_h // 2, 0)
        end_x = min(new_x + new_w, image.shape[1])
        end_y = min(new_y + new_h, image.shape[0])

        cell = image[new_y:end_y, new_x:end_x]
        cv2.imwrite(f"purple_cells/wbc_{cell_num}.png", cell)
        cv2.rectangle(annotated, (new_x, new_y), (end_x, end_y), (0, 255, 0), 2)
        cell_num += 1

    #output_dir = "/content/annotated_results"
    #os.makedirs(output_dir, exist_ok=True)
    #filename = os.path.basename(image_path)
    #output_path = os.path.join(output_dir, os.path.splitext(filename)[0] + "_annotated.png")

    output_path = "/content/output.png"
    cv2.imwrite(output_path, annotated)
    return cell_num

In [ ]:
def predict_from_csv_and_return_results(csv_path, device="cuda" if torch.cuda.is_available() else "cpu"):
    # Load CSV

    label_encoder = joblib.load("/content/label_encoder.pkl")
    num_classes = len(label_encoder.classes_)

    vlm_model = VLMClassifier(num_classes=num_classes).to(device)
    vlm_model.load_state_dict(torch.load("/content/best_vlm_model.pth", map_location=device))
    vlm_model.eval()

    # Load CLIP model
    clip_model, _ = clip.load("ViT-B/32", device=device)
    clip_model.eval()


    df = pd.read_csv(csv_path)

    # Preprocessing for CLIP
    image_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.48145466, 0.4578275, 0.40821073),
                             (0.26862954, 0.26130258, 0.27577711))
    ])

    image_paths = []
    predictions = []

    vlm_model.eval()

    with torch.no_grad():
        for idx, row in df.iterrows():
            image_path = row["Image"]
            img = Image.open(image_path).convert("RGB")
            img_tensor = image_transform(img).unsqueeze(0).to(device)

            # Encode image using CLIP
            image_features = clip_model.encode_image(img_tensor).float()

            # Tabular data
            tabular_tensor = torch.tensor([[row["Count"], row["Mean"], row["Max"], row["Std"]]], dtype=torch.float).to(device)

            # Predict
            output = vlm_model(image_features, tabular_tensor)
            pred_class_idx = torch.argmax(output, dim=1).item()
            pred_class_name = label_encoder.inverse_transform([pred_class_idx])[0]

            image_paths.append(image_path)
            predictions.append(pred_class_name)

    result_df = pd.DataFrame({"Image": image_paths, "Predicted": predictions})
    return result_df


In [ ]:
def preprocess_patch(path, target_size):
    img = image.load_img(path, target_size=target_size)
    arr = image.img_to_array(img) / 255.0
    return np.expand_dims(arr, axis=0)

Folder = '[Malignant] Pro-B'

data_path = "/content/SplitBloodSmearSamples/val/" + Folder + '/'

patch_path = '/content/purple_cells'

#files = glob.glob(f"{data_path}/*")
image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif')
spread_files = [f for f in os.listdir(data_path) if f.lower().endswith(image_extensions)] #Blood Spreads

size = [299, 299]
patch_preds = []

data = []

for BloodSpread in tqdm(spread_files, desc="Processing Blood Smears"):

    #path = os.path.basename(os.path.dirname(file))
    m = 0;
    b = 0;
    patch_preds = []
    clean_up()

    full_path = os.path.abspath(BloodSpread)
    relative_path = data_path + os.path.relpath(full_path, '/content')
    print(relative_path)

    fn_patching(relative_path)

    image_files = [f for f in os.listdir(patch_path) if f.lower().endswith(image_extensions)]
    count = len(image_files)
    if count == 0:
        continue

    for image_file in image_files:
        image_path = os.path.join(patch_path, image_file)

        # Preprocess for each model
        img_eff = preprocess_patch(image_path, size)
        img_inc = preprocess_patch(image_path, size)

        # Predict using both models
        prob_eff = model_eff.predict(img_eff)
        prob_inc = model_inc.predict(img_inc)

        # Ensemble: average
        ensemble_prob = (prob_eff + prob_inc) / 2
        patch_preds.append(ensemble_prob)

    # Final patch-level scores
    patch_preds = np.array(patch_preds)

    # Aggregate for smear-level inference
    mean_score = np.mean(patch_preds)
    max_score = np.max(patch_preds)
    std_score = np.std(patch_preds)


    data.append({'Actual': Folder,'Image': relative_path, 'Count': count, 'Mean': mean_score, 'Max': max_score, 'Std': std_score })

# Create DataFrame
df = pd.DataFrame(data)
df.to_csv('Data.csv', index=False)
resultdf = predict_from_csv_and_return_results('Data.csv')
resultdf.to_csv('Result.csv', index=False)

#files.download('Result.csv')



Processing Blood Smears:   0%|          | 0/119 [00:00<?, ?it/s]

/content/SplitBloodSmearSamples/val/[Malignant] Pro-B/Snap_398.jpg
246
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 506ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 358ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 512ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 337ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 484ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 492ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 464ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 485ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 370ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 482ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 349ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 694ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 633ms/step


Processing Blood Smears:   1%|          | 1/119 [00:10<21:13, 10.79s/it]

/content/SplitBloodSmearSamples/val/[Malignant] Pro-B/Snap_313.jpg
228
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 880ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 349ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 501ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 496ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 466ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 357ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 480ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 699ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 529ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step


Processing Blood Smears:   2%|▏         | 2/119 [00:19<19:12,  9.85s/it]

...

Processing Blood Smears:  99%|█████████▉| 118/119 [19:16<00:08,  8.11s/it]

/content/SplitBloodSmearSamples/val/[Malignant] Pro-B/Snap_294.jpg
236
272.0
229.5
82.0
206.0
154.5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 497ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 557ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 649ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 886ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 612ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 491ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 324ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 491ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 480ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 352ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 473ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 352ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 498ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 364ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 496ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 358ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 491ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 325ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 484ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 397ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 957ms/step
1/1

Processing Blood Smears: 100%|██████████| 119/119 [19:39<00:00,  9.91s/it]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈





TESTING FROM HERE



TESTING FROM HERE



TESTING FROM HERE



TESTING FROM HERE



✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈✈

In [ ]:
#TESTING

#fn_patching_old("/content/SplitBloodSmearSamples/test/Benign/Sap_013 (20).jpg")
fn_patching("/content/SplitBloodSmearSamples/train/Benign/Snap_040 (2).jpg")

244
0.11449412415959


7

In [ ]:
from time import sleep

data_path = "/content/test/"

image_extensions = ('.jpg')
spread_files = [f for f in os.listdir(data_path) if f.lower().endswith(image_extensions)] #Blood Spreads

for BloodSpread in tqdm(spread_files, desc="Testing Blood Smear patching"):

    full_path = os.path.abspath(BloodSpread)
    relative_path = data_path + os.path.relpath(full_path, '/content')
    print(relative_path)

    fn_patching(relative_path)
    clean_up()



Testing Blood Smear patching:   2%|▏         | 1/45 [00:00<00:07,  6.26it/s]

/content/test/Snap_674.jpg
242
0.3010752688172043
3.9047619047619047
/content/test/Snap_068 - Copy.jpg
215


Testing Blood Smear patching:   7%|▋         | 3/45 [00:00<00:05,  7.47it/s]

/content/test/Snap_345.jpg
236
/content/test/Sap_148 (103).jpg
241


Testing Blood Smear patching:  13%|█▎        | 6/45 [00:00<00:04,  8.83it/s]

...

Testing Blood Smear patching:  96%|█████████▌| 43/45 [00:07<00:00,  4.65it/s]

/content/test/Snap_191.jpg
236
/content/test/Snap_166 - Copy.jpg
232


Testing Blood Smear patching:  98%|█████████▊| 44/45 [00:07<00:00,  4.56it/s]

/content/test/Snap_125 - Copy.jpg
231
262.5
326.0


Testing Blood Smear patching: 100%|██████████| 45/45 [00:07<00:00,  5.78it/s]


In [ ]:
#OUTPUT CELLS

#print(df.head(50))
#print(df.tail(20))
from google.colab import files
files.download('output.png')
# Example: filter where prediction is NOT 0.0
#filtered_df = df[df['Prediction'] != 0.0]
#print(filtered_df)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#DONT RUN THIS, THIS IS NOT BEING USED

!pip install roboflow
!pip install inference supervision

from roboflow import Roboflow
rf = Roboflow(api_key="uqDT2nutyqtmEyox0VWN")
project = rf.workspace("smack-lab").project("white-blood-cells-detection")
version = project.version(6)
dataset = version.download("yolov5-obb")




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 6.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 4.7 MB/s eta 0

loading Roboflow workspace...
loading Roboflow project...


In [ ]:
!pwd
!cd /content/
!zip -r patching.zip annotated_results/
!cd ..
!pwd

/content
updating: annotated_results/ (stored 0%)
updating: annotated_results/Sap_148 (105)_annotated.png (deflated 0%)
updating: annotated_results/Snap_047_annotated.png (deflated 0%)
updating: annotated_results/Snap_212 - Copy_annotated.png (deflated 0%)
updating: annotated_results/Snap_059_annotated.png (deflated 1%)
updating: annotated_results/Snap_706_annotated.png (deflated 1%)
updating: annotated_results/Snap_345_annotated.png (deflated 1%)
updating: annotated_results/Snap_672_annotated.png (deflated 1%)
updating: annotated_results/Snap_153_annotated.png (deflated 0%)
updating: annotated_results/Snap_684_annotated.png (deflated 0%)
updating: annotated_results/Snap_650_annotated.png (deflated 0%)
updating: annotated_results/Snap_028 - Copy_annotated.png (deflated 0%)
updating: annotated_results/Snap_072_annotated.png (deflated 0%)
updating: annotated_results/Snap_772_annotated.png (deflated 0%)
updating: annotated_results/Snap_068 - Copy_annotated.png (deflated 0%)
updating: anno

In [ ]:
#DONT RUN THIS, NOT BEING USED

import requests
import json

# Replace with your actual API key, project ID, and model version
api_key = "uqDT2nutyqtmEyox0VWN"
project_id = "white-blood-cells-detection"
model_version = 6

#image
image_path = "/content/SplitBloodSmearSamples/train/Benign/Snap_137.jpg"

# Construct the API endpoint
url = "https://detect.roboflow.com/white-blood-cells-detection/6?api_key=uqDT2nutyqtmEyox0VWN&confidence=0.02"

# Open the image file
with open(image_path, "rb") as image_file:
    image_data = image_file.read()

# Send the POST request
response = requests.post(url, files={"file": image_data})

# Check the response
if response.status_code == 200:
    result = response.json()
    print(json.dumps(result, indent=2))
else:
    print(f"Request failed with status code {response.status_code}: {response.text}")


{
  "inference_id": "6edf3faf-453c-45ec-9e26-cd482fea6d22",
  "time": 0.2769145029997162,
  "image": {
    "width": 1024,
    "height": 768
  },
  "predictions": [
    {
      "x": 564.0,
      "y": 220.0,
      "width": 144.0,
      "height": 138.0,
      "confidence": 0.908223032951355,
      "class": "wbc",
      "class_id": 0,
      "detection_id": "d099820b-e774-474e-889e-5765d8f7249d"
    },
    {
      "x": 655.5,
      "y": 282.5,
      "width": 67.0,
      "height": 59.0,
      "confidence": 0.8907078504562378,
      "class": "wbc",
      "class_id": 0,
      "detection_id": "11486f71-194f-4f63-ae51-030dfcf0e825"
    },
    {
      "x": 650.0,
      "y": 442.0,
      "width": 86.0,
      "height": 114.0,
      "confidence": 0.8799165487289429,
      "class": "wbc",
      "class_id": 0,
      "detection_id": "d09bb87c-0b22-4d70-b00a-1879cd031ced"
    },
    {
      "x": 567.5,
      "y": 512.5,
      "width": 75.0,
      "height": 79.0,
      "confidence": 0.858709454536438,
  

In [ ]:
import os
import shutil

def move_jpgs(source_folder, destination_folder):
    """
    Move all JPG files from source_folder to destination_folder
    """
    # Create destination folder if it doesn't exist
    os.makedirs(destination_folder, exist_ok=True)

    # Get all files in source folder
    files = os.listdir(source_folder)

    # Filter and move JPG files
    moved_count = 0
    for file in files:
        if file.lower().endswith(('.jpg', '.jpeg')):
            source_path = os.path.join(source_folder, file)
            dest_path = os.path.join(destination_folder, file)

            # Move the file
            shutil.move(source_path, dest_path)
            moved_count += 1
            print(f"Moved: {file}")

    print(f"\nDone! Moved {moved_count} JPG files.")

# Example usage:
source = "/content"  # Replace with your source folder path
destination = "/content/test"  # Replace with your destination folder path
move_jpgs(source, destination)

Moved: Snap_674.jpg
Moved: Snap_068 - Copy.jpg
Moved: Snap_345.jpg
Moved: Sap_148 (103).jpg
Moved: Snap_049.jpg
Moved: Snap_100.jpg
Moved: Snap_741.jpg
Moved: Snap_785.jpg
Moved: Snap_772.jpg
Moved: Sap_148 (139).jpg
Moved: Snap_749.jpg
Moved: Snap_023 (3).jpg
Moved: Snap_646.jpg
Moved: Snap_650.jpg
Moved: Snap_167.jpg
Moved: Snap_022 (2).jpg
Moved: Snap_732.jpg
Moved: Snap_101.jpg
Moved: Snap_072.jpg
Moved: Snap_047.jpg
Moved: Snap_111 - Copy.jpg
Moved: Snap_810.jpg
Moved: Snap_820.jpg
Moved: Snap_103 (2).jpg
Moved: Snap_672.jpg
Moved: Snap_109 (2).jpg
Moved: Snap_043.jpg
Moved: Snap_706.jpg
Moved: Snap_068.jpg
Moved: Sap_148 (105).jpg
Moved: Snap_212.jpg
Moved: Snap_099 - Copy.jpg
Moved: Snap_101 - Copy.jpg
Moved: Snap_704.jpg
Moved: Snap_212 - Copy.jpg
Moved: Snap_153.jpg
Moved: Snap_059.jpg
Moved: Snap_620.jpg
Moved: Snap_028 - Copy.jpg
Moved: Snap_655.jpg
Moved: Snap_506.jpg
Moved: Snap_684.jpg
Moved: Snap_191.jpg
Moved: Snap_166 - Copy.jpg
Moved: Snap_125 - Copy.jpg

Done! Moved 

In [ ]:
def normalize_lighting(image, target_bg=230, strength=1.0):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    # Identify probable background pixels: high V, low S
    bg_mask = (v > 180) & (s < 50)
    bg_pixels = v[bg_mask]

    if len(bg_pixels) < 50:
        # Fallback to your corner logic if not enough pixels found
        h_, w_ = v.shape
        corners = np.concatenate([
            v[0:50, 0:50].flatten(),
            v[0:50, w_-50:w_].flatten(),
            v[h_-50:h_, 0:50].flatten(),
            v[h_-50:h_, w_-50:w_].flatten()
        ])
        bg_val = int(np.median(corners))
    else:
        bg_val = int(np.median(bg_pixels))

    print("Estimated background V:", bg_val)

    # Only brighten if darker than target
    if bg_val < target_bg - 10:
        delta = int((target_bg - bg_val) * strength)
    else:
        delta = 0

    v_int = v.astype(np.int16)
    v_new = np.clip(v_int + delta, 0, 255).astype(np.uint8)

    hsv_normalized = cv2.merge([h, s, v_new])
    return cv2.cvtColor(hsv_normalized, cv2.COLOR_HSV2BGR)


In [ ]:
def normalize_lighting(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    h_, w_ = v.shape
    corners = np.concatenate([
        v[0:50, 0:50].flatten(),
        v[0:50, w_-50:w_].flatten(),
        v[h_-50:h_, 0:50].flatten(),
        v[h_-50:h_, w_-50:w_].flatten()
    ])
    bg_val = int(np.median(corners))
    target_bg = 230
    print(bg_val)
    if bg_val < target_bg - 10:
        delta = target_bg - bg_val
    else:
        delta = 0  # no need to brighten if already bright

    # Safely add delta after converting to int16
    v_int = v.astype(np.int16)
    v_new = np.clip(v_int + delta, 0, 255).astype(np.uint8)

    hsv_normalized = cv2.merge([h, s, v_new])
    return cv2.cvtColor(hsv_normalized, cv2.COLOR_HSV2BGR)

def fn_patching(image_path):
    image = cv2.imread(image_path)
    image = normalize_lighting(image)  # <-- Lighting fix added here
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    mask1 = cv2.inRange(hsv, np.array([120, 40, 40]), np.array([135, 255, 255]))
    #mask2 = cv2.inRange(hsv, np.array([116, 90, 60]), np.array([165, 255, 255]))
    mask2 = cv2.inRange(hsv, np.array([118, 45, 80]), np.array([170, 140, 255]))

    main_mask = cv2.bitwise_or(mask1, mask2)

    exclude_lower = np.array([118, 20, 220])
    exclude_upper = np.array([130, 70, 255])
    exclude_mask = cv2.inRange(hsv, exclude_lower, exclude_upper)

    mask = cv2.bitwise_and(main_mask, cv2.bitwise_not(exclude_mask))

    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.dilate(mask, kernel, iterations=1)

    contours_pre, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    sure_bg = cv2.dilate(mask, kernel, iterations=3)
    dist_transform = cv2.distanceTransform(mask, cv2.DIST_L2, 5)
    ret, sure_fg = cv2.threshold(dist_transform, 0.3 * dist_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)

    ret, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1
    markers[unknown == 255] = 0

    wshed_img = image.copy()
    markers = cv2.watershed(wshed_img, markers)
    wshed_mask = np.zeros_like(mask)
    wshed_mask[markers > 1] = 255

    contours_post, _ = cv2.findContours(wshed_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    final_contours = list(contours_post)
    for c in contours_pre:
        if len(c) == 0:
            continue
        M = cv2.moments(c)
        if M["m00"] == 0:
            continue
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        keep = True
        for p in contours_post:
            if cv2.pointPolygonTest(p, (cx, cy), False) >= 0:
                keep = False
                break
        if keep:
            final_contours.append(c)

    os.makedirs("purple_cells", exist_ok=True)
    annotated = wshed_img.copy()
    cell_num = 0
    scale = 1.3

    for contour in final_contours:
        area = cv2.contourArea(contour)
        if area < 500 or area > 23000:
            continue

        x, y, w, h = cv2.boundingRect(contour)
        perimeter = cv2.arcLength(contour, True)
        if perimeter == 0:
            continue

        circularity = 4 * np.pi * (area / (perimeter ** 2))
        if circularity < 0.4:
            continue

        aspect_ratio = float(w) / h
        if aspect_ratio < 0.6 or aspect_ratio > 2.8:
            continue

        hull = cv2.convexHull(contour)
        hull_area = cv2.contourArea(hull)
        if hull_area == 0:
            continue

        mask_roi = np.zeros(mask.shape, dtype=np.uint8)
        cv2.drawContours(mask_roi, [contour], -1, 255, -1)
        mean_val = cv2.mean(image, mask=mask_roi)
        blue, green, red = mean_val[:3]
        if not (blue > red + 10 and blue > green - 10):
            continue

        cx = x + w // 2
        cy = y + h // 2
        new_w = int(w * scale)
        new_h = int(h * scale)
        new_x = max(cx - new_w // 2, 0)
        new_y = max(cy - new_h // 2, 0)
        end_x = min(new_x + new_w, image.shape[1])
        end_y = min(new_y + new_h, image.shape[0])

        cell = image[new_y:end_y, new_x:end_x]
        cv2.imwrite(f"purple_cells/wbc_{cell_num}.png", cell)
        cv2.rectangle(annotated, (new_x, new_y), (end_x, end_y), (0, 255, 0), 2)
        cell_num += 1

    cv2.imwrite("annotated_wbcs_enhanced.png", annotated)
    return cell_num



In [ ]:
def fn_patching_old(image_path):

  image = cv2.imread(image_path)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale

  # Thresholding (Adjust threshold as needed)
  _, mask = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV) # Adjust threshold value

  # Clean up the mask
  kernel = np.ones((3, 3), np.uint8)
  mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
  mask = cv2.erode(mask, kernel, iterations=2)
  mask = cv2.dilate(mask, kernel, iterations=2)

  # Find contours
  contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # Output setup
  os.makedirs("purple_cells", exist_ok=True)
  annotated = gray.copy()
  cell_num = 0

  for contour in contours:
      area = cv2.contourArea(contour)
      if area < 300 or area > 18000:
          continue

      x, y, w, h = cv2.boundingRect(contour)
      aspect_ratio = float(w) / h
      if aspect_ratio < 0.5 or aspect_ratio > 2.8:
          continue

      hull = cv2.convexHull(contour)
      hull_area = cv2.contourArea(hull)
      if hull_area == 0:
          continue
      solidity = float(area) / hull_area
      if solidity < 0.8:
          continue
      # Expand the bounding box
      expansion_pixels = 5  # Adjust as needed
      x_expanded = max(0, x - expansion_pixels)
      y_expanded = max(0, y - expansion_pixels)
      w_expanded = w + 2 * expansion_pixels
      h_expanded = h + 2 * expansion_pixels

      # Create the border region
      border_region = image[y_expanded:y_expanded + h_expanded, x_expanded:x_expanded + w_expanded]

      # Analyze pixel values (Grayscale)
      gray_border = cv2.cvtColor(border_region, cv2.COLOR_BGR2GRAY)

      # Threshold for light pixels
      light_threshold = 200  # Adjust as needed
      light_pixels = gray_border > light_threshold

      # Calculate percentage of light pixels
      light_pixel_percentage = np.sum(light_pixels) / gray_border.size

      # Exclusion threshold
      exclusion_percentage = 0.4  # Adjust as needed

      if light_pixel_percentage > exclusion_percentage:
          # Cell has too much light border, exclude it
          continue

      # Save cropped cell
      cell = image[y:floor(y+h*1.2), x:floor(x+w*1.2)]
      cv2.imwrite(f"purple_cells/wbc_{cell_num}.png", cell)

      # Annotate image
      cv2.rectangle(annotated, (x, y), (x + w, y + h), (0, 255, 0), 2)
      cell_num += 1

  cv2.imwrite("annotated_wbcs_grey.png", annotated)
  return cell_num

  # Save annotated image
  #print(f"{cell_num} WBCs extracted and saved.")